# PyTorch Basics and Gradient Descent

## PyTorch Basics: Tensors and Gradients 

PyTorch is a Python-Python based scientific computing pakage. It's mainly used for applications such as computer vision and natural language processing. The flexibility of PyTorch allow easy integration of new data types and algorithms, and the framework is also efficient and scalable. It was designed to minimize the number of compuations required and to be compatible with a variety of hardware architectures. Two broad purpose of PyTorch are:

* A replacement for NumPy to use the power of GPUs and other accelerators.
* An automatic differentiation library that is useful to implement neural networks.


### Tensors

A tensor is a number, vector. matrix or any n-dimentional array. These are the fundamental data structures in deep learning, which are very similar to arrays and matrics, with which we can efficiently perform mathematical operations on large sets of data. A tensor can be represented as a matrix, but also as a vector, a scalar, or a higher- dimensional array. 

To make it easier to visualize, we can think of a tensor as simple array containing scalars or other arrays. On PyTorch, a tensor is a structure very similar to a **ndarry**, with the differnce that they are capable of running on a GPU, which dramatically speeds up the computational process. 

Let's create a tensor with a single number:

In [1]:
import torch 

In [2]:
t1 = torch.tensor(4.)
t1 

tensor(4.)

`4.` is a shorthand for `4.0`. It is used to indicate to PyTorch that we want to create a floating point number. We can verify it by checking the `dtype` attribte of our tensor.

In [3]:
t1.dtype

torch.float32

Let's create a slightly more compelx tensors:


In [4]:
#Vector
t2 = torch.tensor([1., 2, 3, 4 ])
t2

tensor([1., 2., 3., 4.])

In [5]:
#Matrix
t3 = torch.tensor([[5., 6],
                  [7, 9 ],
                   [9, 10]
                  ])
t3

tensor([[ 5.,  6.],
        [ 7.,  9.],
        [ 9., 10.]])

In [6]:
#3-dimentional array 
t4 = torch.tensor([[[11., 12, 13 ],
                    [13, 14, 15]],
                  [[15, 16, 17],
                  [18, 17, 19]]
                  ])
t4

tensor([[[11., 12., 13.],
         [13., 14., 15.]],

        [[15., 16., 17.],
         [18., 17., 19.]]])

Tensors can have any number of dimensions, and differnt lengths along each dimension. We can inspect the length along each dimension using the `.shape` property of a tensor.

In [7]:
print(t1)
t1.shape

tensor(4.)


torch.Size([])

In [8]:
print(t2)
t2.shape

tensor([1., 2., 3., 4.])


torch.Size([4])

In [9]:
print(t3)
t3.shape

tensor([[ 5.,  6.],
        [ 7.,  9.],
        [ 9., 10.]])


torch.Size([3, 2])

In [10]:
print(t4)
t4.shape

tensor([[[11., 12., 13.],
         [13., 14., 15.]],

        [[15., 16., 17.],
         [18., 17., 19.]]])


torch.Size([2, 2, 3])

### Tensor operations and gradients

We can combine tensors with the usual arthemtic operations. Let's look an example:

In [11]:
# Create tensors,
x = torch.tensor(3.)
w = torch.tensor(4., requires_grad = True)
b = torch.tensor(5., requires_grad = True)
x, w, b

(tensor(3.), tensor(4., requires_grad=True), tensor(5., requires_grad=True))

We have created 3 tensors `x`, `w`, and `b`, all numbers. `w` and `b`have an additional parameter `requires_grad` set to `True`.

Let's create a new tensor `y` by combining these tensors:


In [12]:
# Arithmetic operations
y = w * x + b
y

tensor(17., grad_fn=<AddBackward0>)

As expected, `y` is a tensor with the value `17`. What makes PyTorch special is that we can automatically compute the derivative of `y` w.r.t the tensors that have `requires_grad` set to `True` i.e w and b. To compute the derivatives, we can call the `.backward` method on our result `y`. 

In [13]:
# Compute derivatives
y.backward()

The derivates of `y` w.r.t the input tensor are stored in the `.grad` property of the respective tensors.

In [14]:
# Display gradients
print('dy/dx:', x.grad)
print('dy/dw:', w.grad)
print('dy/db:', b.grad)

dy/dx: None
dy/dw: tensor(3.)
dy/db: tensor(1.)


As expected, `dy/dw` has the same values as `x`i.e `3`. and `dy/db` has the value `1`. Note that `x.grad` is `None`, because `x` does not have `requires_grad` set to `True`.

The "grad" in `w.grad` stands for gradient, which is another term for derivative, used mainly when dealing with matrices.

### Interoperability with Numpy 

Numpy is a popular open source library used for mathematical and scientific computing in Python. It ensables efficient operations on large multi-dimensional arrays, and has a large ecosystem of suporting libraries:
* Matplotlib for plotting and visualization 
* OpenCV for image and video processing 
* Pandas for file I/O and data analysis
instead of reinventing the wheel, PyTorch interopertes really well with Numpy to leverage its existing ecosystem of tools and libraries.

Here's how we create an array in Numpy:


In [15]:
import numpy as np 

In [16]:
x = np.array([[1, 2], [3, 4.]])
x

array([[1., 2.],
       [3., 4.]])

We can convert a Numpy array to a PyTorch tensor using `torch.from_numpy`.

In [17]:
# COnvert the numpy array to a torch tensor.
y = torch.from_numpy(x)
y

tensor([[1., 2.],
        [3., 4.]], dtype=torch.float64)

Let's verify that the numpy array and torch tensor have similar data types.

In [18]:
x.dtype, y.dtype

(dtype('float64'), torch.float64)

We can convert a PyTorch tensor to a Numpy array using the `.numpy` method of a tensor.

In [19]:
#Convert a torch tensor to a numpy array 

z = y.numpy()
z

array([[1., 2.],
       [3., 4.]])

The interoperability between PyTorch and Numpy is really important because most datasets you'll work with will likely be read and preprocessed a Numpy arrays.

## Gradient Descent and Linear Regression with PyTorch 

### Introduction to Linear Regression 

Linear Regression is one of the basic algorithms in machine learning. Linear Regression establishes a linear relationship between input features (X) and output labels (y).

In linear regression, each output label is expressed as a linear function of input features which uses weights and biases. These weights and biases are the model parameters that are initialized randomly but then get updated through each cycle of training/learning through the dataset. Training the model and updating the parameters after going through a single iteration of training data is known as one epoch. So we have to train the model for several epochs so that weights and biases can learn the linear relationship between the input features and output labels.

Now let's understand it in detail with the help of an example, We will create a model that predicts crop yields for apples and oranges (*target variables*) by looking at the average temperature, rainfall, and humidity (*input variables or features*) in a region. Here's the training data:

![linear-regression-training-data](https://i.imgur.com/6Ujttb4.png)

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:

![linear-regression-graph](https://i.imgur.com/4DJ9f8X.png)

The *learning* part of linear regression is to figure out a set of weights `w11, w12,... w23, b1 and b2` using the training data to make accurate predictions for new data. The *learned* weights will be used to predict the yeilds for apples and oranges in a new region using the average temperature, rainfall, and humidity for that region.

We will train our model by adjusiting the weights slightly many times to make better predictions, using an optimization technique called **gradient descent**. Let's begin by importing Numpy and PyTorch.

In [20]:
import numpy as np 
import torch 

### Training data

We can represent the training data using two matrices: `inputs` and `targets`, each with one row per observation, and one column per variable.

In [21]:
#Input (temp, rainffall, humidity)

inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]],
                   dtype='float32')

In [22]:
#Targets (apples, oranges)

targets = np.array([[56, 70],
                   [81, 101],
                   [119, 133],
                   [22, 37],
                   [103, 119]],
                   dtype='float32')

We have seperated the input and target variables because we will operate on them seperately. Also, we have created numpy arrays, because this typically how we work with taining data: read some CSV files as numpy arrays, do some processing and then convert them to PyTorch tensors.

Now, let's convert the arrays to PyTorch tensors.

In [23]:
#Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


### Linear regression model from scratch 

The weights and biases(`w11, w12,... w23, b1 & b2`) can also be represented as matrices, initialized as random values. The first row of `w` and the first element of `b` are used to predict the first target variable,i.e., yield of apples, and similarly, the second for oranges.




In [24]:
# Weights and biases

w = torch.randn(2,3, requires_grad= True)
b = torch.rand(2, requires_grad=True)
print(w)
print(b)

tensor([[ 0.3218, -0.4742,  0.1290],
        [-1.5642,  0.6180,  0.4446]], requires_grad=True)
tensor([0.0146, 0.0753], requires_grad=True)


`torch.randn` creates a tensor with the given shape, with elements picked randomly from a normal distribution with mean 0 and standard deviation 1.

Our *model* is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

![matrix-mult](https://i.imgur.com/WGXLFvA.png)

We can define the model as follows:

In [25]:
def model(x):
    return x @ w.t() +b 

`@` represents matrix multiplication in PyTorch, and the `.t` method return the transpose of a tensor. The matrix obtained by passin the input data into the model is a set of predictions for the targets vraibales. 

In [26]:
#Generate predictions
preds = model(inputs)
print(preds)

tensor([[  -2.7221,  -53.5889],
        [  -4.1801,  -59.4300],
        [ -28.0543,  -27.4153],
        [  17.2162, -116.4494],
        [ -14.2788,  -17.4061]], grad_fn=<AddBackward0>)


Let's compare the predictions of our model with the actual targets.

In [27]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

Here, we can see a big difference between our model's predictions and the actual targets because we've initialized our model with random weights and biases. Obviously, we can't expect a randomly initialized model to just work.

### Loss function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets using the following method:

* Calculate the difference between the two matrices (`preds` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [28]:
#MSE loss
def mse(t1, t2):
    diff = t1 -t2
    return torch.sum(diff*diff)/diff.numel()

`torch.sum` returns the sum of all the elements in a tensor. The `.numel` method of a tensor returns the number of elements in a tensor. Let's compute the mean squared error for the current predictions of our model.

In [29]:
# Compute loss
loss = mse(preds, targets)
loss

tensor(15500.4473, grad_fn=<DivBackward0>)

Here’s how we can interpret the result: *On average, each element in the prediction differs from the actual target by the square root of the loss*. And that’s pretty bad, considering the numbers we are trying to predict are themselves in the range 50–200. The result is called the *loss* because it indicates how bad the model is at predicting the target variables. It represents information loss in the model: the lower the loss, the better the model.

### Introduction to Gradient Descen

The gradient descent algorithm is one of the most popular techniques for training deep neural networks. It has many applications in fields such as computer vision, speech recognition, and natural language processing. While the idea of gradient descent has been around for decades, it’s only recently that it’s been applied to applications related to deep learning.

Gradient descent is an iterative optimization method used to find the minimum of an objective function by updating values iteratively on each step. With each iteration, it takes small steps towards the desired direction until convergence, or a stop criterion is met.

### Compute gradients

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases because they have `requires_grad` set to `True`. We'll see how this is useful in just a moment.

In [30]:
# Compute gradients
loss.backward()

The gradient are stored in the `.grad` property of the respective tensors. Note that the derivtive of the loss w.r.t the weights matrix is itself a matrix with the same dimensions.

In [31]:
# Gradient for weights
print(w)
print(w.grad)

tensor([[ 0.3218, -0.4742,  0.1290],
        [-1.5642,  0.6180,  0.4446]], requires_grad=True)
tensor([[ -6682.4023,  -8519.9941,  -4978.4482],
        [-12528.2197, -12717.4512,  -8022.3965]])


### Adjust weights and biases to reduce the loss

The loss is a [quadratic function](https://en.wikipedia.org/wiki/Quadratic_function) of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. An important insight from calculus is that the gradient indicates the rate of change of the loss, i.e., the loss function's [slope](https://en.wikipedia.org/wiki/Slope) w.r.t. the weights and biases.

If a gradient element is **positive**:

* **increasing** the weight element's value slightly will **increase** the loss
* **decreasing** the weight element's value slightly will **decrease** the loss

![postive-gradient](https://i.imgur.com/WLzJ4xP.png)

If a gradient element is **negative**:

* **increasing** the weight element's value slightly will **decrease** the loss
* **decreasing** the weight element's value slightly will **increase** the loss

![negative=gradient](https://i.imgur.com/dvG2fxU.png)

The increase or decrease in the loss by changing a weight element is proportional to the gradient of the loss w.r.t. that element. This observation forms the basis of _the gradient descent_ optimization algorithm that we'll use to improve our model (by _descending_ along the _gradient_).

We can subtract from each weight element a small quantity proportional to the derivative of the loss w.r.t. that element to reduce the loss slightly.

In [32]:
w
w.grad

tensor([[ -6682.4023,  -8519.9941,  -4978.4482],
        [-12528.2197, -12717.4512,  -8022.3965]])

In [33]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

We multiply the gradients with a very small number (`10^-5` in this case) to ensure that we don't modify the weights by a very large amount. We want to take a small step in the downhill direction of the gradient, not a giant leap. This number is called the *learning rate* of the algorithm. 

We use `torch.no_grad` to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases.

In [34]:
# Let's verify that the loss is actually lower
loss = mse(preds, targets)

In [35]:
loss

tensor(15500.4473, grad_fn=<DivBackward0>)

In [36]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


### Train the model using gradient descent

As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can _train_ the model using the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

Let's implement the above step by step.

In [37]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 10.0060, -32.4715],
        [ 12.5855, -31.7021],
        [ -7.9355,   5.1801],
        [ 29.5387, -95.2323],
        [  1.9970,   9.0643]], grad_fn=<AddBackward0>)

In [38]:
# Calculate the loss 
loss = mse(preds, targets)
loss

tensor(10718.6465, grad_fn=<DivBackward0>)

In [39]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[ -5365.3853,  -7096.7090,  -4101.9604],
        [-10349.9814, -10382.2129,  -6580.1729]])
tensor([ -66.9617, -121.0323])


Let's update the weights and biases using the gradients computed above.

In [40]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad* 1e-5
    w.grad.zero_()
    b.grad.zero_()

Let's take a look at the new weights and baises.

In [41]:
print(w)
print(b)

tensor([[ 0.4422, -0.3180,  0.2198],
        [-1.3354,  0.8489,  0.5907]], requires_grad=True)
tensor([0.0161, 0.0780], requires_grad=True)


with the new weights and biases, the model should have a lowerloss.

In [42]:
# Calculate loss 
preds = model(inputs)
loss = mse(preds, targets)
loss

tensor(7492.8218, grad_fn=<DivBackward0>)

We have already achieved a significant reduction in the loss merely by adjusting the weights and biases slightly using gradient descent.

### Train for multiple epochs

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an _epoch_. Let's train the model for 100 epochs.

In [43]:
# Train for 100 epochs

for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()


Once again, let's verify that the loss is now lower:

In [44]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
loss

tensor(239.8171, grad_fn=<DivBackward0>)

In [45]:
# Train for more 100 epochs

for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

Once again, let's verify that the loss is now lower:

In [46]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
loss

tensor(68.5915, grad_fn=<DivBackward0>)

The loss is now much lower than its initial value. Let's look at the model's predictions and compare them with the targets.

In [47]:
# Predictions
preds

tensor([[ 60.4624,  68.6376],
        [ 83.5747,  98.8334],
        [110.2814, 139.8232],
        [ 40.1457,  27.1456],
        [ 93.1775, 121.7190]], grad_fn=<AddBackward0>)

In [48]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

The predictions are now much close to the target variables. We can get even better results by training for a few more epochs. 

## Linear regression using PyTorch built-ins

We've implemented linear regression & gradient descent model using some basic tensor operations. However, since this is a common pattern in deep learning, PyTorch provides several built-in functions and classes to make it easy to create and train models with just a few lines of code.

Let's begin by importing the `torch.nn` package from PyTorch, which contains utility classes for building neural networks.

In [50]:
import torch.nn as nn

As before, we represent the inputs and targets and matrices.

In [51]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])


We are using 15 training examples to illustrate how to work with large datasets in small batches. 

### Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [53]:
from torch.utils.data import TensorDataset

In [54]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[:4]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.]]))

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation (`[:4]` in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [55]:
from torch.utils.data import DataLoader

In [58]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle = True)

We can use the data loader in a `for` loop. Let's look at an example

In [60]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 91.,  88.,  64.],
        [ 68.,  97.,  70.],
        [ 73.,  67.,  43.],
        [ 87., 135.,  57.],
        [101.,  44.,  37.]])
tensor([[ 81., 101.],
        [102., 120.],
        [ 56.,  70.],
        [118., 134.],
        [ 21.,  38.]])


In each iteration, the data loader returns one batch of data with the given batch size. If `shuffle` is set to `True`, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

### nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [61]:
# Define model
model = nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.0835,  0.4807, -0.5032],
        [ 0.2007,  0.0087,  0.0737]], requires_grad=True)
Parameter containing:
tensor([ 0.0411, -0.5110], requires_grad=True)


PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [62]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[ 0.0835,  0.4807, -0.5032],
         [ 0.2007,  0.0087,  0.0737]], requires_grad=True),
 Parameter containing:
 tensor([ 0.0411, -0.5110], requires_grad=True)]

We can use the model to generate predictions in the same way as before.

In [63]:
# Generate predictions
preds = model(inputs)
preds

tensor([[16.7043, 17.8931],
        [17.7342, 23.2366],
        [42.5337, 22.3921],
        [10.6063, 23.0620],
        [16.7246, 19.3332],
        [16.3070, 18.0851],
        [16.7502, 23.3016],
        [42.1140, 22.6665],
        [11.0036, 22.8700],
        [16.1379, 19.2062],
        [15.7203, 17.9581],
        [17.3369, 23.4286],
        [43.5177, 22.3270],
        [11.1930, 23.1890],
        [17.1218, 19.1412]], grad_fn=<AddmmBackward0>)

### Loss Function

Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [65]:
# Import nn. functional
import torch.nn.functional as F

The `nn.functional` package contains many useful loss functions and several other utilities. 

In [66]:
# Define loss function 
loss_fn = F.mse_loss

Let's compute the loss for the current predictions of our model.

In [67]:
loss = loss_fn(model(inputs), targets)
loss

tensor(5003.3584, grad_fn=<MseLossBackward0>)

### Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD is short for "stochastic gradient descent". The term _stochastic_ indicates that samples are selected in random batches instead of as a single group.

In [68]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Note that `model.parameters()` is passed as an argument to `optim.SGD` so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

### Train the model

We are now ready to train the model. We'll follow the same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data instead of processing the entire training data in every iteration. Let's define a utility function `fit` that trains the model for a given number of epochs.

In [69]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    #Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb, yb in train_dl:
            
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradient to zero 
            
            opt.zero_grad()
            
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}' .format(epoch+1, num_epochs, loss.item()))

Some things to note above:

* We use the data loader defined earlier to get batches of data for every iteration.

* Instead of updating parameters (weights and biases) manually, we use `opt.step` to perform the update and `opt.zero_grad` to reset the gradients to zero.

* We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. `loss.item` returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [70]:
fit(150, model, loss_fn, opt, train_dl)

Epoch [10/150], Loss: 629.7085
Epoch [20/150], Loss: 503.9662
Epoch [30/150], Loss: 275.3478
Epoch [40/150], Loss: 152.7263
Epoch [50/150], Loss: 118.1317
Epoch [60/150], Loss: 99.7144
Epoch [70/150], Loss: 31.3561
Epoch [80/150], Loss: 65.3587
Epoch [90/150], Loss: 82.6656
Epoch [100/150], Loss: 49.7842
Epoch [110/150], Loss: 47.7007
Epoch [120/150], Loss: 32.4631
Epoch [130/150], Loss: 38.1313
Epoch [140/150], Loss: 48.7792
Epoch [150/150], Loss: 44.0983


In [71]:
fit(150, model, loss_fn, opt, train_dl)

Epoch [10/150], Loss: 20.9967
Epoch [20/150], Loss: 9.9297
Epoch [30/150], Loss: 22.3360
Epoch [40/150], Loss: 16.0499
Epoch [50/150], Loss: 15.6234
Epoch [60/150], Loss: 27.4651
Epoch [70/150], Loss: 19.2965
Epoch [80/150], Loss: 12.5596
Epoch [90/150], Loss: 14.5484
Epoch [100/150], Loss: 15.6047
Epoch [110/150], Loss: 9.5797
Epoch [120/150], Loss: 22.9910
Epoch [130/150], Loss: 20.1393
Epoch [140/150], Loss: 18.3676
Epoch [150/150], Loss: 14.3038


Let's generate predictions using our model and verify that they're close to our targets.

In [72]:
# Generate predictions
preds = model(inputs)

In [73]:
preds

tensor([[ 57.3446,  70.5460],
        [ 79.1291,  99.0933],
        [124.4024, 136.0761],
        [ 22.5561,  38.7825],
        [ 95.7866, 115.2688],
        [ 56.0365,  69.4417],
        [ 78.4386,  98.9045],
        [124.3982, 136.5293],
        [ 23.8641,  39.8868],
        [ 96.4042, 116.1842],
        [ 56.6541,  70.3572],
        [ 77.8210,  97.9891],
        [125.0929, 136.2649],
        [ 21.9385,  37.8671],
        [ 97.0947, 116.3730]], grad_fn=<AddmmBackward0>)

In [74]:
# COmpare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

Indeed, the predictions are quite close to our targets. We have a trained a reasonably good model to predict crop yields for apples and oranges by looking at the average temperature, rainfall, and humidity in a region. We can use it to make predictions of crop yields for new regions by passing a batch containing a single row of input.

In [75]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[53.0408, 67.2755]], grad_fn=<AddmmBackward0>)

The predicted yield of apples is 53.0 tons per hectare, and that of oranges is 67.27 tons per hectare.

## Machine Learning vs. Classical Programming

The approach we've taken in this tutorial is very different from programming as you might know it. Usually, we write programs that take some inputs, perform some operations, and return a result. 

However, in this notebook, we've defined a "model" that assumes a specific relationship between the inputs and the outputs, expressed using some unknown parameters (weights & biases). We then show the model some know inputs and outputs and _train_ the model to come up with good values for the unknown parameters. Once trained, the model can be used to compute the outputs for new inputs.

This paradigm of programming is known as _machine learning_, where we use data to figure out the relationship between inputs and outputs. _Deep learning_ is a branch of machine learning that uses matrix operations, non-linear activation functions and gradient descent to build and train models. Andrej Karpathy, the director of AI at Tesla Motors, has written a great blog post on this topics, titled [Software 2.0](https://medium.com/@karpathy/software-2-0-a64152b37c35).

This picture from book [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python) by Francois Chollet captures the difference between classical programming and machine learning:

![](https://i.imgur.com/oJEQe7k.png)

Keep this picture in mind as you work through the next few tutorials. 
